## 타이탄 텍스트 임베딩 V1 벤치마크 : 
## MTEB 데이터 세트를 사용하는 Amazon Bedrock의 새로운 최신 임베딩 모델
https://github.com/aws-samples/amazon-bedrock-samples/tree/main/multimodal/Titan/embeddings/v2

임베딩은 다양한 자연어 처리 애플리케이션에 필수적인 요소로, 최적의 성능을 위해서는 임베딩의 품질이 매우 중요합니다. 임베딩은 일반적으로 지식 기반에서 텍스트 데이터를 고밀도 벡터로 표현하여 효율적인 유사도 검색 및 검색을 가능하게 하는 데 사용됩니다. 검색 증강 생성(RAG)에서 임베딩은 말뭉치에서 관련 구절을 검색하여 언어 모델이 정보에 기반한 지식 기반 응답을 생성할 수 있도록 컨텍스트를 제공하는 데 사용됩니다. 또한 임베딩은 사용자 선호도, 항목 특성 및 과거 상호작용을 벡터로 표현하여 사용자 행동과 항목 임베딩을 기반으로 개인화된 추천을 위한 유사성을 계산함으로써 개인화 및 추천 시스템에서 핵심적인 역할을 합니다. 점진적으로 품질이 개선된 새로운 임베딩 모델이 출시됨에 따라 조직은 컴퓨팅 리소스, 데이터 전처리, 통합 노력, 비즈니스 지표에 영향을 미치는 예상 성능 향상과 같은 요소를 고려하여 업그레이드 관련 비용과 잠재적 이점을 비교 평가해야 합니다.

**MTEB에 유용한 링크:**
1. https://github.com/embeddings-benchmark/mteb

2. https://huggingface.co/blog/mteb

#### 텍스트는 어떻게 벡터로 변환되나요?
일반적인 접근 방식은 전체 문장에 대해 문맥화된 임베딩을 제공할 수 있는 모델을 사용하는 것입니다. 이러한 모델은 트랜스포머와 같은 딥러닝 아키텍처를 기반으로 하며, 문장의 문맥 정보와 단어 간의 관계를 보다 효과적으로 포착할 수 있습니다.

![임베딩 모델](./images/vector_embedding.png)

시맨틱 검색 외에도 임베딩을 사용하여 검색 증강 생성(RAG)을 통해 프롬프트를 보강하여 보다 정확한 결과를 얻을 수 있지만, 이를 사용하려면 벡터 기능이 있는 데이터베이스에 임베딩을 저장해야 합니다.

![임베딩 모델](./images/vector_db.jpg)


2023년 9월, Amazon은 단일 단어, 구문 또는 대용량 문서와 같은 텍스트 입력을 고차원 숫자 벡터 표현으로 변환하는 다국어 텍스트 임베딩 모델인 Amazon Titan 텍스트 임베딩 V1의 출시를 발표했습니다.  그 이후 수천 명의 고객이 25개 이상의 언어를 지원하고 최대 8,192개의 토큰을 입력할 수 있으며 1,536차원의 벡터를 출력하는 첫 번째 버전의 모델을 사용했습니다. 오늘 저희는 유연한 출력 임베딩 모델을 도입하여 이를 한 단계 더 발전시켰습니다. Amazon Titan 텍스트 임베딩 V2 모델은 100개 이상의 언어를 지원하며, 가변 차원 출력을 허용하여 임베딩을 저장하는 데 필요한 크기를 줄여 비용을 절감할 수 있습니다. 출력 크기는 256, 512, 1024입니다. 이 모델은 다국어 데이터 및 사용 사례에서 우수한 성능을 발휘하도록 설계되었습니다.


#### Few installs

벤치마크를 위해 MTEB와 같은 몇 가지 라이브러리를 설치해야 합니다. 단순히 모델을 사용하려면 boto3만 있으면 됩니다.

In [2]:
#!pip install -U boto3  huggingface_hub mteb

#### 모델에 연결할 Boto3 클라이언트

이 옵션은 선택 사항이며 **boto3.client('bedrock')** 와 같은 간단한 클라이언트를 사용하여 호출할 수 있습니다. 이렇게 하면 기본 프로필이 Bedrock에 액세스하는 것으로 가정합니다.

In [10]:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(assumed_role: Optional[str] = None, region: Optional[str] = 'us-east-1',runtime: Optional[bool] = True,external_id=None, ep_url=None):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides 
    """
    target_region = region

    print(f"Create new client\n  Using region: {target_region}:external_id={external_id}: ")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        if external_id:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
                ExternalId=external_id
            )
        else:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
            )
        print(f"Using role: {assumed_role} ... sts::successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    if ep_url:
        bedrock_client = session.client(service_name=service_name,config=retry_config,endpoint_url = ep_url, **client_kwargs )
    else:
        bedrock_client = session.client(service_name=service_name,config=retry_config, **client_kwargs )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

#### Helper Class to connect and run the embeddings

이것은 프로덕션 사용 코드가 아니라 참조용 코드입니다.

In [11]:
import json
import boto3

class TitanEmbeddings(object):
    accept = "application/json"
    content_type = "application/json"
    
    def __init__(self, model_id="amazon.titan-embed-text-v1", boto3_client=None, region_name='us-east-1'):
        
        if boto3_client:
            self.bedrock_boto3 = boto3_client
        else:
            # self.bedrock_boto3 = boto3.client(service_name='bedrock-runtime')
            self.bedrock_boto3 = boto3.client(
                service_name='bedrock-runtime', 
                region_name=region_name, 
            )
        self.model_id = model_id

    def __call__(self, text):
        """
        Returns Titan Embeddings

        Args:
            text (str): text to embed
            dimensions (int): Number of output dimensions.
            normalize (bool): Whether to return the normalized embedding or not.

        Return:
            List[float]: Embedding
            
        """

        body = json.dumps({
            "inputText": text,
        })

        response = self.bedrock_boto3.invoke_model(
            body=body, modelId=self.model_id, accept=self.accept, contentType=self.content_type
        )

        response_body = json.loads(response.get('body').read())

        return response_body['embedding']


### Test connection to bedrock

베드락 boto client를 사용합니다. 가정할 역할(role)을 사용하는 경우 해당 역할을 전달하세요. 프로필이 설정되어 있다면 운영 환경 변수 AWS_PROFILE에 설정하여 활용하세요.

In [12]:
import json
import os
import sys

import boto3

boto3_bedrock_runtime = get_bedrock_client() #boto3.client('bedrock')

bedrock_embeddings = TitanEmbeddings(model_id="amazon.titan-embed-text-v1", boto3_client=boto3_bedrock_runtime)
bedrock_embeddings

Create new client
  Using region: us-east-1:external_id=None: 
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 임베딩 생성

현재 API를 통해 amazon.titan-embed-text-v2를 임베딩 모델로 사용할 수 있습니다. 입력 텍스트 크기는 8k 토큰이며 출력 벡터 길이는 256, 512 또는 1024 중 하나를 선택할 수 있습니다.

텍스트 임베딩 모델을 사용하려면 InvokeModel API 작업 또는 Python SDK를 사용합니다. InvokeModel을 사용하여 지정된 모델에서 입력 텍스트의 벡터 표현을 검색합니다.

Input

```

{
    "inputText": text,
    "dimensions": dimensions, # range from 256 , 512, 1024
    "normalize": normalize
}

Output

{
    "embedding": []
}
```

#### 벡터의 정규화 

정규화는 단위 길이 또는 크기가 1이 되도록 스케일을 조정하는 과정입니다. 벡터 연산 중에 모든 벡터가 동일한 스케일을 가지며 동등하게 기여하도록 하여 일부 벡터가 더 큰 크기로 인해 다른 벡터를 지배하는 것을 방지하는 데 유용합니다.

#### 언제 정규화해야 할까요?
검색, RAG 등과 같은 대부분의 사용 사례에서 이 옵션을 기본값으로 사용합니다.

#### 정규화하지 말아야 할 경우: 
일반적으로 정규화는 모든 사용 사례에서 작동하지만, 분류 또는 엔티티 추출과 같은 특정 사용 사례에 대해서는 실험이 필요합니다.


In [13]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."
#amazon.titan-embed-text-v1
modelId = "amazon.titan-embed-text-v1"  # 
accept = "application/json"
contentType = "application/json"


sample_model_input={
    "inputText": prompt_data,
    #"dimensions": 256,
    #"normalize": True
}

body = json.dumps(sample_model_input)

response = boto3_bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

embedding = response_body.get("embedding")
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")


The embedding vector has 1536 values
[0.16601562, 0.23632812, 0.703125, '...', 0.26953125, -0.609375, -0.55078125]


#### Use the Helper class to show the same results

In [14]:
embedding= bedrock_embeddings(text=prompt_data)
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

The embedding vector has 1536 values
[0.16601562, 0.23632812, 0.703125, '...', 0.26953125, -0.609375, -0.55078125]


### Optional MTEB 테스트

자체 MTEB 테스트를 만들려면 아래 코드를 시작점 및 샘플로 사용할 수 있습니다. 이 코드는 프로덕션 코드가 아니라는 점에 유의하세요. 자세한 내용은 MTEB의 깃허브 위치(https://github.com/embeddings-benchmark/mteb/tree/main) 를 참조하세요.

In [15]:
# amazon.titan-embed-text-v1:0
import numpy as np
import json
import os
import sys
import time
import torch

import boto3

class TitanV1Model():
    def __init__(self) -> None:

        self.br_embeddings = None     
        self._init_connection()
        
        print(f"TitanV2Model:__init__::ready:to:Invoke:::successful::") 
    
    def _init_connection(self, dim=256):
        boto3_bedrock_runtime = get_bedrock_client() #boto3.client('bedrock')

        self.br_embeddings = TitanEmbeddings(model_id="amazon.titan-embed-text-v1", boto3_client=boto3_bedrock_runtime)
        self.dim = dim
        
    def process_dict_text(self, single_text_dict):
        """ **IMPORTANT** CHANGE this Code to be tuned to your data set and use this -- DO NOT USE THIS AS IS. refer to  https://github.com/embeddings-benchmark/mteb/blob/main/mteb/abstasks/AbsTaskRetrieval.py as an example. Please goover this Git hub in detail"""
        single_text = [str(key).strip() + " " + str(val).strip() if 'title' in key.lower() else str(val).strip() for key, val in single_text_dict.items()]
        return " ".join(single_text)[:30000]
        
    def reorg_text(self, single_text):
        
        """ **IMPORTANT** CHANGE this Code to be tuned to your data set and use this -- DO NOT USE THIS AS IS. refer to  https://github.com/embeddings-benchmark/mteb/blob/main/mteb/abstasks/AbsTaskRetrieval.py as an example. Please goover this Git hub in detail"""
        if isinstance(single_text, dict):
            single_text = self.process_dict_text(single_text)
        single_text = "0" if not single_text else single_text 
        # check for json -- 
        try:
            single_text_dict = json.loads(single_text)
            single_text = self.process_dict_text(single_text_dict)
        except:
            pass
        return single_text
    
    def invoke_model(self, text_list: list[str]):
        """ **IMPORTANT** CHANGE this Code to be tuned to your data set and use this -- DO NOT USE THIS AS IS. refer to  https://github.com/embeddings-benchmark/mteb/blob/main/mteb/abstasks/AbsTaskRetrieval.py as an example. Please goover this Git hub in detail"""
        list_embeddings = []
        
        for single_text in text_list:
            single_text = self.reorg_text(single_text)
            single_embed = bedrock_embeddings(text=single_text)
            list_embeddings.append(single_embed)

        return list_embeddings

    def reshape_titan_embeddings(self, query_embeddings: np.ndarray, **kwargs) -> list[np.ndarray]:
        # - use this to re shape your embeddings as needed
        return query_embeddings # 
        
        
    def encode(self, queries: list[str], **kwargs) -> list[np.ndarray] | list[torch.Tensor] : # - | list[torch.Tensor] 
        """
        Returns a list of embeddings for the given sentences.
        Args:
            queries: List of sentences to encode

        Returns:
            List of embeddings for the given sentences
        """
        
        embedding_list = self.invoke_model(queries)
        return self.reshape_titan_embeddings(np.array(embedding_list))

    
    def encode_queries(self, queries: list[str], **kwargs) -> list[np.ndarray] | list[torch.Tensor] : # - | list[torch.Tensor] 
        """
        Returns a list of embeddings for the given sentences.
        Args:
            queries: List of sentences to encode

        Returns:
            List of embeddings for the given sentences
        """
        
        embedding_list = self.invoke_model(queries)
        return self.reshape_titan_embeddings(np.array(embedding_list))


    def encode_corpus(self, corpus: list[str] | list[dict[str, str]], **kwargs) -> list[np.ndarray] | list[torch.Tensor] : #- | list[torch.Tensor]:
        """
        Returns a list of embeddings for the given sentences.
        Args:
            corpus: List of sentences to encode
                or list of dictionaries with keys "title" and "text"

        Returns:
            List of embeddings for the given sentences
        """
        
        embedding_list = self.invoke_model(corpus)
        return self.reshape_titan_embeddings(np.array(embedding_list))


### Sample of one of the classfication tasks which can be run

In [30]:
from mteb import MTEB
from sentence_transformers import SentenceTransformer

# Define the sentence-transformers model name
#model_name = "average_word_embeddings_komninos"
# or directly from huggingface:
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "amazon.titan-embed-text-v1"
model = TitanV2Model()
evaluation = MTEB(tasks=["Banking77Classification"])
results = evaluation.run(model, output_folder=f"results/{model_name}")

Create new client
  Using region: us-east-1:external_id=None: 
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
TitanV2Model:__init__::ready:to:Invoke:::successful::


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- Banking77Classification, s2s

In [17]:
#XPQARetrieval
from mteb import MTEB
from sentence_transformers import SentenceTransformer

# Define the sentence-transformers model name
#model_name = "average_word_embeddings_komninos"
# or directly from huggingface:
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "amazon.titan-embed-text-v1"
model = TitanV1Model()
evaluation = MTEB(tasks=["STS17"],task_langs=["kor"])
results = evaluation.run(model, output_folder=f"results/{model_name}")

Create new client
  Using region: us-east-1:external_id=None: 
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
TitanV2Model:__init__::ready:to:Invoke:::successful::


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

STS

- STS17, s2s, crosslingual 11 / 11 pairs

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for mteb/sts17-crosslingual-sts contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mteb/sts17-crosslingual-sts
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [18]:
import json
import pandas as pd

# JSON 파일 읽어오기
with open('results/amazon.titan-embed-text-v1/STS17.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 'en-en'과 'ko-ko'의 'cos_sim', 'euclidean', 'manhattan' 값 추출
en_en_data = data['test']['en-en']
ko_ko_data = data['test']['ko-ko']

# 데이터 프레임 생성
df = pd.DataFrame({
    'Metric': ['cos_sim', 'euclidean', 'manhattan'],
    'en-en Pearson': [en_en_data['cos_sim']['pearson'], en_en_data['euclidean']['pearson'], en_en_data['manhattan']['pearson']],
    'en-en Spearman': [en_en_data['cos_sim']['spearman'], en_en_data['euclidean']['spearman'], en_en_data['manhattan']['spearman']],
    'ko-ko Pearson': [ko_ko_data['cos_sim']['pearson'], ko_ko_data['euclidean']['pearson'], ko_ko_data['manhattan']['pearson']],
    'ko-ko Spearman': [ko_ko_data['cos_sim']['spearman'], ko_ko_data['euclidean']['spearman'], ko_ko_data['manhattan']['spearman']]
})

# 표 출력
print(df)


      Metric  en-en Pearson  en-en Spearman  ko-ko Pearson  ko-ko Spearman
0    cos_sim       0.786702        0.809049       0.641521        0.645054
1  euclidean       0.746642        0.764455       0.642402        0.647367
2  manhattan       0.747067        0.763869       0.641621        0.646169


### Please refer to this link [for Amazon Titan models](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-models.html) 